In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
songs = pd.read_csv("df_audio_features_5000")
songs.columns=songs.columns.str.strip()
songs.drop_duplicates(inplace=True)
songs.dropna(inplace=True)
scaler = MinMaxScaler()
num_cols = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
songs[num_cols] = scaler.fit_transform(songs[num_cols])
kmeans = KMeans(n_clusters=5, init='k-means++', random_state=42)
kmeans.fit(songs[num_cols])
songs['cluster'] = kmeans.labels_

In [ ]:
songs=songs.sample(30)
songs.groupby("cluster").count()

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd

client_id = 'YOUR_CLIENT_ID'  
client_secret = 'YOUR_CLIENT_SECRET'  
redirect_uri = 'YOUR_URI'  
scope = 'playlist-modify-public'

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri, scope=scope))

for cluster_id in songs['cluster'].unique():

    cluster_songs = songs[songs['cluster'] == cluster_id]

    playlist_name = f'Cluster {cluster_id} Playlist'
    playlist_description = f'A playlist with representative songs from cluster {cluster_id}'
    playlist = sp.user_playlist_create(user='YOUR_USERNAME', name=playlist_name, public=True, description=playlist_description)  
    playlist_id = playlist['id']
    
    song_ids = cluster_songs['id']
    song_ids = [song_id.strip() for song_id in song_ids]
    track_uris = [f'spotify:track:{song_id}' for song_id in song_ids]
    sp.playlist_add_items(playlist_id=playlist_id, items=track_uris)
    print(f'Added {len(track_uris)} songs to {playlist_name}')